# ML-box Avocado

#### Goal :

- Create a ML model using Auto-sklearn for the Avocado dataset
- Get RMSE over the predictions of these model

#### Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics
from math import sqrt
from mlbox.preprocessing import Reader
from mlbox.preprocessing import Drift_thresholder
from mlbox.optimisation import make_scorer
from mlbox.optimisation import Optimiser
from mlbox.prediction import Predictor

Paths to the train set and the test set.

In [2]:
paths = ["../../Data/avocado_price/processed/train.csv","../../Data/avocado_price/processed/x_test.csv"]

Name of the feature to predict. This columns should only be present in the train set.

In [3]:
target_name = "AveragePrice"

Reading and cleaning all files. Declare a reader for csv files

In [4]:
rd = Reader(sep=',')

Return a dictionnary containing three entries:
- dict["train"] contains training samples withtout target columns
- dict["test"] contains testing elements withtout target columns
- dict["target"] contains target columns for training samples.

In [5]:
data = rd.train_test_split(paths, target_name)


reading csv : train.csv ...
cleaning data ...
CPU time: 0.3984375 seconds

reading csv : x_test.csv ...
cleaning data ...
CPU time: 0.1798863410949707 seconds

> Number of common features : 14

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 0
> Number of numerical features: 14
> Number of training samples : 12226
> Number of test samples : 6023

> You have no missing values on train set...

> Task : regression
count    12226.000000
mean       140.106658
std         40.259655
min         44.000000
25%        110.000000
50%        137.000000
75%        166.000000
max        312.000000
Name: AveragePrice, dtype: float64


Removing the drifting variables

In [6]:
dft = Drift_thresholder()
data = dft.fit_transform(data)


computing drifts ...
CPU time: 1.0780882835388184 seconds

> Top 10 drifts

('year', 0.01652095766931816)
('Month', 0.014052133375274112)
('Total Volume', 0.011363754825062689)
('Day', 0.009383030176192797)
('region', 0.007258513221157337)
('type_organic', 0.006209422086088434)
('type_conventional', 0.005249107658505725)
('4225', 0.0039422234659385325)
('Large Bags', 0.003932261120233038)
('Small Bags', 0.002992500014094013)

> Deleted variables : []
> Drift coefficients dumped into directory : save


Tuning

In [7]:
mape = make_scorer(lambda y_true,
                   y_pred: 100*np.sum(
                                      np.abs(y_true-y_pred)/y_true
                                      )/len(y_true),
                   greater_is_better=False,
                   needs_proba=False)

Declare an optimiser

In [8]:
opt = Optimiser(scoring="mean_squared_error", n_folds=3)

/opt/conda/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


Space of hyperparameters

In [9]:
space = {
        'ne__numerical_strategy': {"search": "choice",
                                   "space": [0]},
        'ce__strategy': {"search": "choice",
                         "space": ["label_encoding",
                                   "random_projection",
                                   "entity_embedding"]},
        'fs__threshold': {"search": "uniform",
                          "space": [0.01, 0.3]},
        'est__max_depth': {"search": "choice",
                           "space": [3, 4, 5, 6, 7]}

        }

Optimises hyper-parameters of the whole Pipeline

In [10]:
best = opt.optimise(space,data,40)
print("Final results :" ,opt.evaluate(best, data))

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'entity_embedding'}      
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.22360205192383217}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 3, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:296: UserWarning: Unknown or invalid scoring metric. neg_mean_squared_error is used instead.
  warnings.warn("Unknown or invalid scoring metric. "

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5759007.281884576, tolerance: 1330.8976461472394
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5331342.366964197, tolerance: 1306.6447314685313
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5589147.332537356, tolerance: 1325.4078826156306
  

MEAN SCORE : neg_mean_squared_error = -311.7605332271602
VARIANCE : 10.094937585700304 (fold 1 = -298.0946937094737, fold 2 = -315.0164053211818, fold 3 = -322.17050065082503)
CPU time: 3.6697702407836914 seconds                  
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}    
>>> CA ENCODER :{'strategy': 'entity_embedding'}                               
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.25070684648745273}     
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 3, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent'

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5759007.281884576, tolerance: 1330.8976461472394
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5331342.366964197, tolerance: 1306.6447314685313
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5589147.332537356, tolerance: 1325.4078826156306
  positive)



MEAN SCORE : neg_mean_squared_error = -311.7605332271602                       
VARIANCE : 10.094937585700304 (fold 1 = -298.0946937094737, fold 2 = -315.0164053211818, fold 3 = -322.17050065082503)
CPU time: 3.342552423477173 seconds                                            
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}    
>>> CA ENCODER :{'strategy': 'label_encoding'}                                 
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.24002696950514701}     
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 3, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': Non

...

MEAN SCORE : neg_mean_squared_error = -311.7605332271602                         
VARIANCE : 10.094937585700304 (fold 1 = -298.0946937094737, fold 2 = -315.0164053211818, fold 3 = -322.17050065082503)
CPU time: 2.5643367767333984 seconds                                             
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}      
>>> CA ENCODER :{'strategy': 'entity_embedding'}                                 
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.25306146990786754}       
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 6, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_s

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5759007.281884576, tolerance: 1330.8976461472394
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5331342.366964197, tolerance: 1306.6447314685313
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5589147.332537356, tolerance: 1325.4078826156306
  positive)



MEAN SCORE : neg_mean_squared_error = -174.67717672441282                        
VARIANCE : 5.501864652483698 (fold 1 = -166.9035751198656, fold 2 = -178.27395933465385, fold 3 = -178.85399571871903)
CPU time: 6.229955196380615 seconds                                              
100%|██████████| 40/40 [03:20<00:00,  5.02s/trial, best loss: 170.54364618714683]


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ BEST HYPER-PARAMETERS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

{'ce__strategy': 'entity_embedding', 'est__max_depth': 7, 'fs__threshold': 0.05173766265956951, 'ne__numerical_strategy': 0}

##################################################### testing hyper-parameters... ##########

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5759007.281884576, tolerance: 1330.8976461472394
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5331342.366964197, tolerance: 1306.6447314685313
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5589147.332537356, tolerance: 1325.4078826156306
  positive)



MEAN SCORE : neg_mean_squared_error = -170.54364618714683
VARIANCE : 4.3465765548258 (fold 1 = -164.51930673783417, fold 2 = -174.61392378439146, fold 3 = -172.4977080392149)
CPU time: 6.651137828826904 seconds

Final results : -170.54364618714683


Make a prediction

In [11]:
from mlbox.prediction import *
pred=Predictor()
pred.fit_predict(best,data)


fitting the pipeline ...


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8395496.948463427, tolerance: 1981.4766918043524
  positive)


CPU time: 2.3373866081237793 seconds

> Feature importances dumped into directory : save

predicting...
CPU time: 0.3124966621398926 seconds

> Overview on predictions : 

   AveragePrice_predicted
0              117.372834
1              210.799902
2              140.863435
3              209.266109
4              119.459608
5              131.543681
6              139.468283
7              175.802513
8              121.024673
9               96.659715

dumping predictions into directory : save ...


Getting the predictions and targets

In [12]:
y_pred = pd.read_csv("save/AveragePrice_predictions.csv")
predictions = y_pred.AveragePrice_predicted
y_test = pd.read_csv("../../Data/avocado_price/processed/y_test.csv")

Calculating RMSE

In [13]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print("rmse score:", sqrt(mean_squared_error(y_test, predictions)))

rmse score: 12.225484120626746


## Residual Plot

In [14]:
import matplotlib.pyplot as plt

plt.scatter(predictions, predictions - y_test.AveragePrice, c="grey", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
plt.title("Residual Plot")
plt.show()